In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./raw_data/data.csv", encoding="ISO-8859-1")

In [5]:
# DATA CLEANING & TRANSFORMATION

raw_data_length = len(df)

# ---------------------------- Drop nan ------------------------------------------------------
print(f"The dataset contains {df.isnull().sum().sum()} duplicate rows that need to be removed.")
df = df.dropna(subset=["Description", "CustomerID"])
assert df.isnull().sum().sum() == 0
# --------------------------------------------------------------------------------------------
# ---------------------------- Drop duplciates -----------------------------------------------

print(f"The dataset contains {df.duplicated().sum()} duplicate rows that need to be removed.")
df.drop_duplicates(inplace=True)
assert df.duplicated().sum() == 0
# --------------------------------------------------------------------------------------------

# --------------------------- Canceled transaction cut ---------------------------------------
df["Transaction_Status"] = np.where(df["InvoiceNo"].astype(str).str.startswith("C"), "Cancelled", "Completed").copy()
df = df[df["Transaction_Status"] == "Completed"]
# --------------------------------------------------------------------------------------------

# -------------------------- delete anomalous stock codes ------------------------------------
unique_stock_codes = df['StockCode'].unique()
numeric_char_counts_in_unique_codes = pd.Series(unique_stock_codes).apply(lambda x: sum(c.isdigit() for c in str(x))).value_counts()
anomalous_stock_codes = [code for code in unique_stock_codes if sum(c.isdigit() for c in str(code)) in (0, 1)]
df = df[~df['StockCode'].isin(anomalous_stock_codes)]
# --------------------------------------------------------------------------------------------

# Resetting the index of the cleaned dataset
df.reset_index(drop=True, inplace=True)

# delete unitprice == 0
df = df[df['UnitPrice'] > 0]

print(f"raw data len: {raw_data_length}; cleaned data len: {len(df)}")

The dataset contains 136534 duplicate rows that need to be removed.
The dataset contains 5225 duplicate rows that need to be removed.
raw data len: 541909; cleaned data len: 391150


In [6]:
# FEATURE ENGINERING

"""
    Recency (R): This metric indicates how recently a customer has made a purchase.
    A lower recency value means the customer has purchased more recently, indicating higher engagement with the brand.

    Frequency (F): This metric signifies how often a customer makes a purchase within a certain period.
    A higher frequency value indicates a customer who interacts with the business more often, suggesting higher loyalty or satisfaction.

    Monetary (M): This metric represents the total amount of money a customer has spent over a certain period.
    Customers who have a higher monetary value have contributed more to the business, indicating their potential high lifetime value.
"""
# --------------------------------- RECENCY ----------------------------------------------------------
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["InvoiceDay"] = df["InvoiceDate"].dt.date

customer_data = df.groupby("CustomerID")["InvoiceDay"].max().reset_index()
most_recent_date = df["InvoiceDay"].max()

customer_data["InvoiceDay"] = pd.to_datetime(customer_data["InvoiceDay"])
most_recent_date = pd.to_datetime(most_recent_date)

customer_data["Days_Since_Last_Purchase"] = (most_recent_date - customer_data["InvoiceDay"]).dt.days
customer_data

customer_data.drop(columns=["InvoiceDay"], inplace=True)

# ------------------------------- FREQ --------------------------------------------------------------

total_transactions = df.groupby("CustomerID")["InvoiceNo"].nunique().reset_index()
total_transactions.rename(columns={"InvoiceNo": "Total_Transactions"}, inplace=True)

total_products_purchased = df.groupby("CustomerID")["Quantity"].sum().reset_index()
total_products_purchased.rename(columns={"Quantity": "Total_Products_Purchased"}, inplace=True)

customer_data = pd.merge(customer_data, total_transactions, on="CustomerID")
customer_data = pd.merge(customer_data, total_products_purchased, on="CustomerID")

# ---------------------------- MONETARY -------------------------------------------------------------

df["Total_Spend"] = df["UnitPrice"] * df["Quantity"]
total_spend = df.groupby("CustomerID")["Total_Spend"].sum().reset_index()

average_transaction_value = total_spend.merge(total_transactions, on="CustomerID")
average_transaction_value["Average_Transaction_Value"] = average_transaction_value["Total_Spend"] / average_transaction_value["Total_Transactions"]

customer_data = pd.merge(customer_data, total_spend, on="CustomerID")
customer_data = pd.merge(customer_data, average_transaction_value[["CustomerID", "Average_Transaction_Value"]], on="CustomerID")

customer_data.head()

# ------------------------- FILTERING --------------------------------------------------------------

customer_data = customer_data[customer_data["Days_Since_Last_Purchase"] < 55]
customer_data = customer_data[(customer_data["Total_Spend"] > 10) & (customer_data["Total_Spend"] < 2000)]
customer_data = customer_data[customer_data["Total_Transactions"] > 3]

customers = customer_data["CustomerID"].to_list()

df = df[df["CustomerID"].isin(customers)]

In [7]:
# to timestamp
df_copy = df.copy()

df_copy["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Timestamp"] = df_copy["InvoiceDate"].astype(np.int64) // 10 ** 9

df = df.sort_values(by="Timestamp", ascending=False)

In [8]:
df.to_csv("./raw_data/prepared.csv")

In [18]:
# reference to .py file -> fetch_img_url.py
# create table with images

item_images = pd.read_csv("./image_item.csv")
item_images = item_images[["item_id", "image_url"]]

items_to_sql = df[["StockCode", "Description"]]

items_to_sql = items_to_sql.rename(columns={"StockCode": "item_id", "Description": "description"})
items_to_sql = items_to_sql.drop_duplicates(subset=["item_id"])

merged = pd.merge(item_images, items_to_sql, on="item_id")
merged = merged[["item_id", "description", "image_url"]]

merged

,item_id,description,image_url
0,22138,BAKING SET 9 PIECE RETROSPOT,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,22367,CHILDRENS APRON SPACEBOY DESIGN,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,22556,PLASTERS IN TIN CIRCUS PARADE,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,22555,PLASTERS IN TIN STRONGMAN,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,22728,ALARM CLOCK BAKELIKE PINK,https://encrypted-tbn0.gstatic.com/images?q=tb...
...,...,...,...
3091,35957,SMALLFOLKART BAUBLE CHRISTMAS DEC,https://encrypted-tbn0.gstatic.com/images?q=tb...
3092,85018D,YULETIDE IMAGES S/6 PAPER BOXES,https://encrypted-tbn0.gstatic.com/images?q=tb...
3093,79067,CORONA MEXICAN TRAY,https://encrypted-tbn0.gstatic.com/images?q=tb...
3094,21911,GARDEN METAL SIGN,https://encrypted-tbn0.gstatic.com/images?q=tb...


In [48]:
# CREATE ITEM TABLE TO SQL

import psycopg2

def execute_many(conn, df, table):
    tuples = [tuple(x) for x in df.values]
    cols = ','.join(list(df.columns))

    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s)" % (table, cols)

    cursor = conn.cursor()

    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    cursor.close()


conn = psycopg2.connect("dbname=postgres_db user=postgres_user password=postgres_password", port=5430)

items_to_sql = df[["StockCode", "Description"]]

items_to_sql = items_to_sql.rename(columns={"StockCode": "item_id", "Description": "description"})
items_to_sql = items_to_sql.drop_duplicates(subset=["item_id"])

execute_many(conn, items_to_sql, "items")

In [114]:
# Prepare to atomic format
df = df[["CustomerID", "StockCode", "Timestamp"]]
df = df.rename(columns={"CustomerID": "user_id:token", "StockCode": "item_id:token", "Timestamp": "timestamp:float"})
df


,user_id:token,item_id:token,timestamp:float
391182,12680.0,22138,1323435000
391168,12680.0,22631,1323435000
391177,12680.0,23256,1323435000
391181,12680.0,23255,1323435000
391180,12680.0,23254,1323435000
...,...,...,...
312,15862.0,22451,1291202460
311,15862.0,22766,1291202460
310,15862.0,22100,1291202460
340,15862.0,22810,1291202460


In [115]:
# writing to recbole atomic file
df.to_csv("dataset/data/data.inter", index=False, sep="\t")